In [1]:
from pathlib import Path
from IFRS import *
from parsing import *
from chunking import chunking_text
import json

/home/hlm/Documents/Mini-RAG/.rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = Path().resolve().parent
raw_path = path / "data" / "raw"

doc_path = path / "data" / "cleaned"
final_path = path/"data"/"metadatas"

In [3]:
global_metadata = []
#Cleaning and saving files
for i in raw_path.iterdir() :
    if "IFRS" in str(i) :

        final_text, final_appendix = global_process_ifrs(i)
        file_name = re.findall(r"IFRS_\d+",str(i))[0]

        path_text = path / "data" / "cleaned" / (file_name+"_main.txt")
        path_appendix = path / "data" / "cleaned" / (file_name+"_appendix.txt")

        with open(path_text, "w", encoding="utf-8") as file :
            file.write(final_text)
        with open(path_appendix, "w", encoding="utf-8") as file :
            file.write(final_appendix)

        # Parsing cleaned files and aggregating metadata
        parsed = parse(final_text,file_name,appendix=False)
        global_metadata.extend(parsed)

        parsed = parse(final_appendix,file_name,appendix=True)
        global_metadata.extend(parsed)

all_chunks = chunking_text(global_metadata, 400, 60, True) 

In [4]:
with open(final_path, "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=2)